# Img only

In [4]:
import os
import pandas as pd
import numpy as np

folder = "./results/img_only/"
results = None
for exp in os.listdir(folder):
    if os.path.isfile(folder+exp):
        continue
    model, _, _, fe = exp.split("_")
    for run_name in os.listdir(folder + exp):
        
        fp = folder + exp + "/" + run_name + "/summary_latest.csv"
        if os.path.isfile(fp):
            df = pd.read_csv(fp)
            # eval_df = pd.read_csv(folder + exp + "/" + run_name + "/baskent_eval_summary.csv")
            if len(df) < 5:
                print(run_name, "has only", len(df), "rows")
            mean_score = df["test_cindex"].mean()
            std_score = df["test_cindex"].std()

            out = {
                "fe":     fe[:3],
                "max_test_cindex": df["test_cindex"].max(),
                "mean_test_cindex": df["test_cindex"].mean(),
                "std_test_cindex": df["test_cindex"].std(),
                # "max_baskent_cindex": eval_df["cindex"].max(),
                # "mean_baskent_cindex": eval_df["cindex"].mean(),
                # "std_baskent_cindex": eval_df["cindex"].std(),
            }
            
            if results is None:
                results = {k:[] for k in out.keys()}
            for k in out.keys():
                results[k].append(out[k])

df = pd.DataFrame(results)
df = df.sort_values("max_test_cindex", ascending=False)
df

run_s1 has only 4 rows
run_s1 has only 4 rows
run_s1 has only 1 rows


,fe,max_test_cindex,mean_test_cindex,std_test_cindex
1,SSL,0.614920,0.604292,0.010078
0,CTP,0.552386,0.545804,0.006744
2,Res,0.540044,0.540044,NaN


# fusion search

In [52]:
import json
import os
import sys
import numpy as np
import pandas as pd
import itertools

def all_param_combinations(params):
    keys = params.keys()
    values = (params[key]["values"] for key in keys)
    return [dict(zip(keys, combination)) for combination in itertools.product(*values)]

def save_run_commands(save_name, params, pages=3, bootstrap=False):
    i = 1
    lines = []
    beginning = True
    os.makedirs(f"./results/run_scripts", exist_ok=True)
    for c in range(len(params)):
        if beginning:
            beginning = False
            
            lines.append("#!/bin/bash")
            lines.append("cd /media/nfs/Desktop/OViTANet/")
            lines.append("source /home/ezgitwo/miniconda3/etc/profile.d/conda.sh")
            lines.append("conda activate torch")
        one_line = f"CUDA_VISIBLE_DEVICES=0 python main.py "
        if bootstrap:
            one_line += "--bootstrap "
        
        for k, v in params[c].items():
            one_line += f"--{k} {v} "
        
        lines.append(one_line)

        if c % int(len(params)//pages+1) == 0 and c > 0:
            print(c)
            with open(f"./results/run_scripts/{save_name}{i}.sh", "w") as f:
                f.write("\n".join(lines))
            i += 1
            lines = []
            beginning = True
    if not beginning:
        with open(f"./results/run_scripts/{save_name}{i}.sh", "w") as f:
            f.write("\n".join(lines))

In [53]:
import os
import pandas as pd
import numpy as np
rm_list = []
folder = "./fs_nn_results_os/VIT_tab_4cls/"
results = None
for run_name in os.listdir(folder):
    if os.path.isfile(folder+run_name):
        continue
    _, _, tab_data = run_name.split("_", 2)  
    
    fp = folder + run_name + "/summary_latest.csv"
    if os.path.isfile(fp):
        test_nb = len(pd.read_csv(folder+run_name+"/splits_0.csv")["test"].dropna().values)
        df = pd.read_csv(fp)
        n = len(df)
        if n < 5:
            print(fp)
        overall_mean_cindex = df['test_cindex'].mean()
        pooled_std = np.sqrt(((df['test_cindex_std']**2).sum())/n)
        ci_95_lower = overall_mean_cindex - 1.96 * (pooled_std / np.sqrt(n))
        ci_95_upper = overall_mean_cindex + 1.96 * (pooled_std / np.sqrt(n))
        
        out = {
            "test_nb": test_nb,
            "tab_data_type": tab_data[:3],
            "tab_data": tab_data,#[4:],
            # "nb_feats": len(tabular_fs[tab_data]),
            "mean": overall_mean_cindex,
            "std": pooled_std,
            "ci_lower": ci_95_lower,
            "ci_upper": ci_95_upper,
            
        }
        if results is None:
            results = {k:[] for k in out.keys()}
        for k in out.keys():
            results[k].append(out[k])
    else:
        rm_list.append(folder + run_name)
        print(fp)

df1 = pd.DataFrame(results)
df1 = df1.sort_values("mean", ascending=False)
df1

,test_nb,tab_data_type,tab_data,mean,std,ci_lower,ci_upper
35,115,cli,cli_collinreduced+univariate,0.652848,0.033876,0.623154,0.682542
11,115,cli,cli_highvar+collinreduced+coxnet1,0.631368,0.038931,0.597244,0.665493
18,115,cli,cli_collinreduced+coxnet2,0.616087,0.034901,0.585495,0.646679
80,57,mut,mut_highvar+univariate+coxnet2,0.598382,0.042502,0.561128,0.635637
20,83,pro,pro_collinreduced+univariate+coxnet1,0.587251,0.048959,0.544336,0.630165
...,...,...,...,...,...,...,...
51,60,rna,rna_collinreduced+univariate,0.404746,0.058465,0.353499,0.455993
83,57,mut,mut_collinreduced,0.404255,0.051329,0.359263,0.449247
71,57,mut,mut_collinreduced+coxnet2,0.397801,0.048214,0.355539,0.440063
54,115,cli,cli_coxnet2,0.387070,0.038539,0.353290,0.420851


In [54]:
top_dfs = []
for dt in df1["tab_data_type"].unique():
    # print(dt, df1["nb_feats"][df1["tab_data_type"] == dt].min())
    top_dfs.append(df1[df1["tab_data_type"] == dt].sort_values("mean", ascending=False).iloc[:1])
top_dfs = pd.concat(top_dfs)
top_dfs

,test_nb,tab_data_type,tab_data,mean,std,ci_lower,ci_upper
35,115,cli,cli_collinreduced+univariate,0.652848,0.033876,0.623154,0.682542
80,57,mut,mut_highvar+univariate+coxnet2,0.598382,0.042502,0.561128,0.635637
20,83,pro,pro_collinreduced+univariate+coxnet1,0.587251,0.048959,0.544336,0.630165
87,60,rna,rna_highvar+coxnet2,0.579440,0.058680,0.528005,0.630875
59,113,dna,dna_highvar+collinreduced+univariate+coxnet2,0.547002,0.042010,0.510179,0.583826
24,112,cnv,cnv_univariate,0.536642,0.041476,0.500286,0.572997


In [55]:
top_tabs = {k: [] for k in ["cli", "pro", "rna", "dna", "cnv", "mut"]}

for g in top_dfs["tab_data"].values:
    # if g[:3] != "dna":
    top_tabs[g[:3]].append(g)
top_tabs = {k: v for k, v in top_tabs.items() if len(v)>0}
for k, v in top_tabs.items():
    print(k, v)
top_combos = []
for k in range(1, len(list(top_tabs.keys()))+1):
    combos = list(itertools.combinations(list(top_tabs.keys()), k))
    for c in combos:
        genes = [top_tabs[g] for g in c]
        gene_pairs = list(itertools.product(*genes))
        gene_pairs = [(",").join(i) for i in gene_pairs]
        # print(gene_pairs)
        top_combos.extend(gene_pairs)
len(top_combos)

cli ['cli_collinreduced+univariate']
pro ['pro_collinreduced+univariate+coxnet1']
rna ['rna_highvar+coxnet2']
dna ['dna_highvar+collinreduced+univariate+coxnet2']
cnv ['cnv_univariate']
mut ['mut_highvar+univariate+coxnet2']


63

In [57]:
save_name = "fusion"
k = 1
parameter_dict = {
    "results_dir": {
        "values": [f"./results/{save_name}_k{k}/"]
    },
    "split_dir": {
        "values": ["tcga_ov"]
    },
    "k": {
        "values": [k]
    },
    # "n_classes": {
    #     "values": [3]
    # },
    "tabular_data":{
        "values": top_combos
    },
    "data_root_dir": {
        "values": [f"/media/nfs/SURV/TCGA_OV/Feats1024/"+fe for fe in ["SSL"]]#, "CTP", "Res"]]
    },
    "mm_fusion": {
        "values": ["concat", "adaptive", "multiply"]
    },
    "mm_fusion_type": {
        "values": ["early", "mid", "late"]
    },
}

params = all_param_combinations(parameter_dict)
print(len(params))
parameter_dict["mm_fusion"]["values"] = ["crossatt"]
parameter_dict["mm_fusion_type"]["values"] = ["mid"]
params += all_param_combinations(parameter_dict)
print(len(params))
parameter_dict["mm_fusion"]["values"] = ["bilinear"]
parameter_dict["mm_fusion_type"]["values"] = ["late"]
params += all_param_combinations(parameter_dict)
print("Total runs: ", len(params))
save_run_commands(save_name, params, pages=2)

63
32


In [58]:
import os
import pandas as pd
import numpy as np
thresholds = [500, 350, 250, 200, 150]
# thresholds = [400, 300, 200, 150, 120]
rm_list = []
folder = f"./results/fusion_k1/VIT_path+tab_4cls_SSLfeats/"
results = {k:[] for k in ["total_nb", "test_nb", "fusion", "fusion_loc", "fusion_type", "test_cindex", "test_loss","cli", "rna", "pro", "cnv", "dna", "mut"]}
small_ds_list = []
for run_name in os.listdir(folder):
    _, _, fusion, tab_data = run_name.split("_", 3) 
    # if tab_data not in top_combos:
    #     continue
    tab_data = tab_data.split(",")
    tab_vars = {k: [i for i in tab_data if i[:3] == k] for k in ["cli", "rna", "pro", "cnv", "dna", "mut"]} 
    for k in tab_vars.keys():
        if len(tab_vars[k]) > 0:
            tab_vars[k] = tab_vars[k][0]
        else:
            tab_vars[k] = None
    
    fp = folder + run_name + "/summary_latest.csv"
    if os.path.isfile(fp):
        split_df = pd.read_csv(folder+run_name+"/splits_0.csv")
        train_nb = len(split_df["train"].dropna().values) + len(split_df["val"].dropna().values)
        test_nb = len(split_df["test"].dropna().values)
        
        df = pd.read_csv(fp)
        score = df["test_cindex"].iloc[0]

        auc = df["test_auc"].iloc[0]
        ibs = df["test_ibs"].iloc[0]
        loss = df["test_loss"].iloc[0]
        
        out = {
            "total_nb": train_nb+test_nb,
            "test_nb": test_nb,
            "fusion": fusion,
            "fusion_loc": fusion.split(",")[0],
            "fusion_type": fusion.split(",")[1],
            # "test_ibs": round(ibs, 4),
            # "test_auc": round(auc, 4),

            "test_cindex": round(score, 4),
            "test_loss": round(loss, 4),
        }
        out.update(tab_vars)
        
        for k in out.keys():
            results[k].append(out[k])
    else:
        print(fp)
        rm_list.append(folder + run_name)


df2 = pd.DataFrame(results)
df2 = df2.sort_values("test_cindex", ascending=False)
def get_category(total):
    for i, threshold in enumerate(thresholds):
        if total > threshold:
            return i+1
    return 6
df2["category"] = df2["total_nb"].apply(get_category)
df2

./results/fusion_k1/VIT_path+tab_4cls_SSLfeats/run_s1_late,bilinear_pro_collinreduced+univariate+coxnet1,rna_highvar+coxnet2,dna_highvar+collinreduced+univariate+coxnet2,mut_highvar+univariate+coxnet2/summary_latest.csv
./results/fusion_k1/VIT_path+tab_4cls_SSLfeats/run_s1_late,concat_cli_collinreduced+univariate,dna_highvar+collinreduced+univariate+coxnet2,cnv_univariate/summary_latest.csv


,total_nb,test_nb,fusion,fusion_loc,fusion_type,test_cindex,test_loss,cli,rna,pro,cnv,dna,mut,category
67,224,46,"late,multiply",late,multiply,0.7274,2.0375,cli_collinreduced+univariate,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,None,4
304,179,34,"late,concat",late,concat,0.6882,1.7907,None,rna_highvar+coxnet2,None,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,mut_highvar+univariate+coxnet2,5
143,205,38,"late,multiply",late,multiply,0.6709,2.8135,None,None,pro_collinreduced+univariate+coxnet1,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,mut_highvar+univariate+coxnet2,4
257,414,83,"late,adaptive",late,adaptive,0.6641,1.2125,None,None,pro_collinreduced+univariate+coxnet1,None,None,None,2
315,574,115,"early,multiply",early,multiply,0.6615,1.2337,cli_collinreduced+univariate,None,None,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,137,28,"late,multiply",late,multiply,0.3333,3.1474,None,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,cnv_univariate,None,mut_highvar+univariate+coxnet2,6
406,139,28,"late,bilinear",late,bilinear,0.3171,1.9780,None,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,None,None,mut_highvar+univariate+coxnet2,6
196,205,38,"early,adaptive",early,adaptive,0.3163,1.6723,None,None,pro_collinreduced+univariate+coxnet1,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,mut_highvar+univariate+coxnet2,4
43,137,28,"late,adaptive",late,adaptive,0.3130,3.6614,cli_collinreduced+univariate,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,mut_highvar+univariate+coxnet2,6


In [59]:
df2[df2["fusion"] == "mid,crossatt"]

,total_nb,test_nb,fusion,fusion_loc,fusion_type,test_cindex,test_loss,cli,rna,pro,cnv,dna,mut,category
219,574,115,"mid,crossatt",mid,crossatt,0.6448,1.2944,cli_collinreduced+univariate,None,None,None,None,None,1
370,139,28,"mid,crossatt",mid,crossatt,0.6423,1.4266,cli_collinreduced+univariate,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,None,None,mut_highvar+univariate+coxnet2,6
180,414,83,"mid,crossatt",mid,crossatt,0.6357,1.2602,None,None,pro_collinreduced+univariate+coxnet1,None,None,None,2
444,414,83,"mid,crossatt",mid,crossatt,0.6316,1.2246,cli_collinreduced+univariate,None,pro_collinreduced+univariate+coxnet1,None,None,None,2
236,402,80,"mid,crossatt",mid,crossatt,0.6212,1.2071,cli_collinreduced+univariate,None,pro_collinreduced+univariate+coxnet1,cnv_univariate,None,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,205,38,"mid,crossatt",mid,crossatt,0.4055,4.0007,cli_collinreduced+univariate,None,pro_collinreduced+univariate+coxnet1,cnv_univariate,None,mut_highvar+univariate+coxnet2,4
553,224,46,"mid,crossatt",mid,crossatt,0.4007,2.3300,None,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,None,4
547,208,38,"mid,crossatt",mid,crossatt,0.3758,1.7874,None,None,pro_collinreduced+univariate+coxnet1,None,None,mut_highvar+univariate+coxnet2,4
317,296,60,"mid,crossatt",mid,crossatt,0.3700,2.2170,None,rna_highvar+coxnet2,None,cnv_univariate,None,None,3


In [46]:
df2_1 = df2[df2["category"] == 1]
df2_1 = df2_1.sort_values("test_cindex", ascending=False).reset_index(drop=True)
print(len(df2_1))

df2_2 = df2[df2["category"] == 2]
df2_2 = df2_2.sort_values("test_cindex", ascending=False).reset_index(drop=True)
print(len(df2_2))

df2_3 = df2[df2["category"] == 3]
df2_3 = df2_3.sort_values("test_cindex", ascending=False).reset_index(drop=True)
print(len(df2_3))

df2_4 = df2[df2["category"] == 4]
df2_4 = df2_4.sort_values("test_cindex", ascending=False).reset_index(drop=True)
print(len(df2_4))

df2_5 = df2[df2["category"] == 5]
df2_5 = df2_5.sort_values("test_cindex", ascending=False).reset_index(drop=True)
print(len(df2_5))

df2_6 = df2[df2["category"] == 6]
df2_6 = df2_6.sort_values("test_cindex", ascending=False).reset_index(drop=True)
print(len(df2_6))

70
79
149
137
80
73


In [47]:
pd.concat([df2_1.iloc[:1], df2_2.iloc[:1], df2_3.iloc[:1], df2_4.iloc[:1], df2_5.iloc[:1], df2_6.iloc[:1]])

,total_nb,test_nb,fusion,fusion_loc,fusion_type,test_cindex,test_loss,cli,rna,pro,cnv,dna,mut,category
0,574,115,"early,multiply",early,multiply,0.6615,1.2337,cli_collinreduced+univariate,None,None,None,None,None,1
0,414,83,"late,adaptive",late,adaptive,0.6641,1.2125,None,None,pro_collinreduced+univariate+coxnet1,None,None,None,2
0,291,59,"early,adaptive",early,adaptive,0.6244,3.3454,None,rna_highvar+coxnet2,None,None,dna_highvar+collinreduced+univariate+coxnet2,None,3
0,224,46,"late,multiply",late,multiply,0.7274,2.0375,cli_collinreduced+univariate,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,None,4
0,179,34,"late,concat",late,concat,0.6882,1.7907,None,rna_highvar+coxnet2,None,cnv_univariate,dna_highvar+collinreduced+univariate+coxnet2,mut_highvar+univariate+coxnet2,5
0,139,28,"mid,concat",mid,concat,0.6504,1.4320,cli_collinreduced+univariate,rna_highvar+coxnet2,pro_collinreduced+univariate+coxnet1,None,None,mut_highvar+univariate+coxnet2,6


In [4]:
import pandas as pd
dfs = []
for subdf in [df2_1, df2_2, df2_3, df2_4, df2_5, df2_6]:
    dfs.append(subdf.groupby("fusion").max()[["test_cindex"]].round(4))
fusion_df = pd.concat(dfs, axis=1)
fusion_df.columns = ["cat1_os", "cat2_os", "cat3_os", "cat4_os", "cat5_os", "cat6_os"]
fusion_df

/tmp/ipykernel_590224/2703850658.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  dfs.append(subdf.groupby("fusion").max()[["test_cindex"]].round(4))
/tmp/ipykernel_590224/2703850658.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  dfs.append(subdf.groupby("fusion").max()[["test_cindex"]].round(4))
/tmp/ipykernel_590224/2703850658.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  dfs.append(subdf.groupby("fusion").max()[["test_cindex"]].round(4))
/tmp/ipykernel_590224/2703850658.py:4: FutureW

,cat1_os,cat2_os,cat3_os,cat4_os,cat5_os,cat6_os
fusion,,,,,,
"early,adaptive",0.5986,0.6473,0.6244,0.5393,0.5511,0.5610
"early,concat",0.6131,0.6479,0.6073,0.5665,0.5565,0.6463
"early,multiply",0.6615,0.6200,0.6003,0.5520,0.5914,0.5285
"late,adaptive",0.6418,0.6641,0.5969,0.5414,0.5806,0.5650
"late,concat",0.6083,0.6525,0.6064,0.6263,0.6882,0.5122
"late,multiply",0.5941,0.6068,0.5594,0.7274,0.5430,0.6382
"mid,adaptive",0.6256,0.6409,0.6002,0.5796,0.6237,0.5935
"mid,concat",0.5996,0.6345,0.6081,0.5690,0.5645,0.6504
"mid,crossatt",0.6448,0.6357,0.5467,0.5987,0.5726,0.6423


In [72]:
# read pkl
import pickle
import pandas as pd

# fp = "./results/VIT_path_4cls_SSLfeats/run_s1/split_latest_test_0_results.pkl"
fp ="./results/fusion_deneme/VIT_tab_4cls/n1_s1_dna_highvar+collinreduced+univariate+coxnet2/split_latest_test_0_results.pkl"
with open(fp, "rb") as f:
    results = pickle.load(f)

results = pd.DataFrame(results).T

results

,case_id,risk,time,event,hazards
TCGA-23-2084,[TCGA-23-2084],[-0.50907546],[49.8],[1.0],"[0.107284315, 0.2614154, 0.47580713, 0.59894174]"
TCGA-23-1110,[TCGA-23-1110],[-0.64598995],[54.47],[0.0],"[0.043318603, 0.111043245, 0.36580083, 0.5597018]"
TCGA-23-1109,[TCGA-23-1109],[-0.68643486],[51.31],[1.0],"[0.044999044, 0.10215882, 0.2891813, 0.4687051]"
TCGA-23-1809,[TCGA-23-1809],[-0.4097925],[0.53],[0.0],"[0.15996794, 0.3770375, 0.60355973, 0.67044824]"
TCGA-25-1628,[TCGA-25-1628],[-0.5749532],[20.6],[1.0],"[0.08936511, 0.21595754, 0.3662196, 0.5078562]"
...,...,...,...,...,...
TCGA-29-1784,[TCGA-29-1784],[-0.7099116],[5.35],[0.0],"[0.05294541, 0.10724519, 0.23827596, 0.37413245]"
TCGA-29-2432,[TCGA-29-2432],[-0.5157806],[21.45],[1.0],"[0.11575207, 0.25023338, 0.45045316, 0.58402157]"
TCGA-09-1661,[TCGA-09-1661],[-0.56990063],[38.4],[1.0],"[0.091928594, 0.20961936, 0.39374754, 0.49740776]"
TCGA-10-0928,[TCGA-10-0928],[-0.686348],[18.5],[1.0],"[0.056388937, 0.10628977, 0.26259783, 0.45871633]"


In [74]:
results["hazards"].iloc[0]

array([0.10728431, 0.2614154 , 0.47580713, 0.59894174], dtype=float32)

In [73]:
results["y_pred"] = [pd.NA] * len(results)
for i in range(len(results)):
    hazards = results.iloc[i]["hazards"]
    print(hazards)
    S = np.cumprod(1-hazards)
    print(S)
        
    indices = (S < 0.5).nonzero()
    print(indices[0])
    y_pred = indices[0][0].item() if len(indices[0]) > 0 else len(S)
    print(y_pred)

[0.10728431 0.2614154  0.47580713 0.59894174]
[0.8927157  0.6593461  0.34562454 0.13861558]
[2 3]
2
[0.0433186  0.11104324 0.36580083 0.5597018 ]
[0.9566814  0.85044837 0.5393536  0.23747642]
[3]
3
[0.04499904 0.10215882 0.2891813  0.4687051 ]
[0.95500094 0.85743916 0.6094838  0.32381564]
[3]
3
[0.15996794 0.3770375  0.60355973 0.67044824]
[0.84003204 0.52330846 0.20746054 0.06836899]
[2 3]
2
[0.08936511 0.21595754 0.3662196  0.5078562 ]
[0.9106349  0.71397644 0.45250425 0.22269717]
[2 3]
2
[0.1016978 0.2793183 0.500459  0.6350518]
[0.8983022  0.64738995 0.32339782 0.11802346]
[2 3]
2
[0.01477934 0.02021557 0.06850165 0.14077945]
[0.9852207 0.9653039 0.899179  0.7725931]
[]
4
[0.14485495 0.34501955 0.5468048  0.6377203 ]
[0.85514504 0.56010324 0.2538361  0.09195967]
[2 3]
2
[0.11333859 0.2687932  0.4498781  0.5455373 ]
[0.8866614  0.64833283 0.3566621  0.16208962]
[2 3]
2
[0.09988189 0.21979417 0.3894374  0.5369209 ]
[0.9001181  0.7022774  0.4287843  0.19856106]
[2 3]
2
[0.10165969 0.2

In [28]:
[np.min(i[0])  for i in  results.loc["attention_weights"].values]

[5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e-06,
 5.873714e

In [20]:
results.drop("attention_weights").to_csv("./results/img_only/VIT_path_4cls_SSLfeats/run_s1/pt_results0.csv")

In [61]:
results.loc["hazards"].values

array([array([0.4299617 , 0.5882531 , 0.36612767, 0.62876445, 0.5373913 ,
              0.4154795 , 0.4887447 , 0.56348485, 0.49730456, 0.5421213 ,
              0.58696663, 0.40056798, 0.57895917, 0.54073226, 0.54692996,
              0.5188667 , 0.41329578, 0.5612134 , 0.37774956, 0.6136078 ,
              0.5641843 , 0.5657604 , 0.48104098, 0.5560827 , 0.6023261 ,
              0.42686862, 0.5010376 , 0.44692934, 0.41113746, 0.45424637,
              0.45858032, 0.45288178, 0.34557757, 0.52375287, 0.34301996,
              0.47844702, 0.44100016, 0.5777101 , 0.53538394, 0.37789616,
              0.5649058 , 0.36184826, 0.48722413, 0.43082878, 0.48154488,
              0.5942557 , 0.6116447 , 0.54278487, 0.42293322, 0.51487184],
             dtype=float32)                                               ,
       array([0.4397135 , 0.5677201 , 0.36416754, 0.640655  , 0.48487303,
              0.4497279 , 0.5003835 , 0.5493194 , 0.4695564 , 0.58059794,
              0.57240105, 0.4092963